In [ ]:
import os
import pathlib
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import SVC
from PIL import Image, ImageOps
from IPython.display import display
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

In [ ]:
def process_image(img_path: str) -> np.array:
    img = Image.open(img_path)
    img = ImageOps.grayscale(img)
    img = img.resize(size=(96, 96))
    img = np.ravel(img) / 255.0 # flaten the img to make computer deal with
    return img

In [ ]:
def process_folder(folder: pathlib.PosixPath) -> pd.DataFrame:
    # We'll store the images here
    processed = []
    
    # For every image in the directory
    for img in folder.iterdir():
        # Ensure JPG
        if img.suffix == '.jpg':
            # Two images failed for whatever reason, so let's just ignore them
            try:
                processed.append(process_image(img_path=str(img)))
            except Exception as _:
                continue
           
    # Convert to pd.DataFrame
    processed = pd.DataFrame(processed)
    # Add a class column - dog or a cat
    processed['class'] = folder.parts[-1]
    
    return processed

In [ ]:
train_cat = process_folder(folder=pathlib.Path.cwd().joinpath(r'D:\study\grade 3\1st term\Selected 1\project\imgsvm\pets\PetImages\cat'))
train_dog = process_folder(folder=pathlib.Path.cwd().joinpath(r'D:\study\grade 3\1st term\Selected 1\project\imgsvm\pets\PetImages\dog'))
df = pd.concat([train_cat, train_dog], axis=0)

with open('train_set.pkl','wb') as f:
    pickle.dump(df,f)

In [ ]:
df.info

In [ ]:
df=shuffle(df).reset_index(drop=True)
#valid_set=shuffle(valid_set).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
X = df.loc[:, df.columns!='class']
Y = df['class']

In [ ]:
Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.15, shuffle=True,random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( X, Y, test_size=0.15, shuffle=True,random_state=42)

In [ ]:
y_train

In [ ]:
# from sklearn.svm import SVC
# svm = SVC(C=100, gamma='auto')
# svm.fit(X_train , y_train)
# y_pred=svm.predict(X_test)

In [ ]:
from sklearn.svm import SVC
svm = SVC(C=100, gamma='auto')
svm.fit(X_train , y_train)
y_pred=svm.predict(X_val)

In [ ]:
%%time
from sklearn.metrics import accuracy_score
accuracy_score(y_val , y_pred)

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix,plot_roc_curve
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val,y_pred)
cm

In [ ]:
import seaborn as sns

ax = sns.heatmap(cm, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import learning_curve
train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(svm, X, Y, cv=30,return_times=True)
plt.plot(train_sizes,np.mean(train_scores,axis=1))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
fig=plot_roc_curve(svm,X_val,y_val)
plt.title("ROC CURVE")
plt.show()

In [ ]:
y_pred_test=svm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred_test)